In [ ]:
%pip install govuk-bank-holidays

In [ ]:
import pandas as pd
from deltalake import write_deltalake
from govuk_bank_holidays.bank_holidays import BankHolidays
from notebookutils import credentials

# Fetch bank holidays data
bank_holidays_client = BankHolidays(locale='en', use_cached_holidays=True)
division = 'england-and-wales'
holidays_list = bank_holidays_client.get_holidays(division=division)
bank_holidays_df = pd.DataFrame(holidays_list).sort_values(by='date').reset_index(drop=True)

# Convert date column to proper date type
bank_holidays_df['date'] = pd.to_datetime(bank_holidays_df['date'], dayfirst=True).dt.date

# Define Delta Lake table path
delta_table_path = 'abfss://Data_Forge@onelake.dfs.fabric.microsoft.com/LH_core_datetime.Lakehouse/Tables/bank_holidays'

# Storage options for authentication
storage_options = {
    "bearer_token": credentials.getToken('storage'),
    "use_fabric_endpoint": "true",
    "allow_unsafe_rename": "true"
}

# Write to Delta Lake
write_deltalake(
    delta_table_path,
    bank_holidays_df,
    mode='overwrite',
    schema_mode='merge',
    engine='rust',
    storage_options=storage_options
)


In [ ]:
from deltalake import DeltaTable

# Define the mounted path to the Delta table
delta_table_path = "/lakehouse/default/Tables/shift_patten"

# Load the Delta table from the mounted path
delta_table = DeltaTable(delta_table_path)

# Retrieve the data as a PyArrow Dataset
pyarrow_dataset = delta_table.to_pyarrow_dataset()

# Convert the PyArrow Dataset to a Table
pyarrow_table = pyarrow_dataset.to_table()

# Convert the PyArrow Table to a Pandas DataFrame
shift_pattern_df = pyarrow_table.to_pandas()

# Display the DataFrame
display(shift_pattern_df)


In [ ]:
from deltalake import DeltaTable

# Define the mounted path to the Delta table
delta_table_path = "/lakehouse/default/Tables/dcc_school_holidays"

# Load the Delta table from the mounted path
delta_table = DeltaTable(delta_table_path)

# Retrieve the data as a PyArrow Dataset
pyarrow_dataset = delta_table.to_pyarrow_dataset()

# Convert the PyArrow Dataset to a Table
pyarrow_table = pyarrow_dataset.to_table()

# Convert the PyArrow Table to a Pandas DataFrame
school_holidays_df = pyarrow_table.to_pandas()

# Display the DataFrame
display(school_holidays_df)


In [ ]:
display(bank_holidays_df)

display(shift_pattern_df)

display(school_holidays_df)

In [ ]:
import pandas as pd

# Define date range
start_date = "2000-04-01"
end_date = "2035-12-31"
dates = pd.date_range(start=start_date, end=end_date, freq='D')

# Base DataFrame
date_dimension_df = pd.DataFrame({'Date': dates})

# Day-level info
date_dimension_df["Day"] = date_dimension_df["Date"].dt.day
date_dimension_df["DayOfWeek"] = date_dimension_df["Date"].dt.dayofweek  # Monday=0
date_dimension_df["DayName_Short"] = date_dimension_df["Date"].dt.strftime('%a')
date_dimension_df["DayName_Long"] = date_dimension_df["Date"].dt.strftime('%A')
date_dimension_df["IsWeekend"] = date_dimension_df["DayOfWeek"].isin([5, 6])  # Saturday=5, Sunday=6
date_dimension_df["DayOfYear"] = date_dimension_df["Date"].dt.dayofyear

# Calendar (CY) info
date_dimension_df["CY_Year"] = date_dimension_df["Date"].dt.year
date_dimension_df["CY_Quarter"] = "Q" + date_dimension_df["Date"].dt.quarter.astype(str)
date_dimension_df["CY_Month"] = date_dimension_df["Date"].dt.month
date_dimension_df["MonthName_Short"] = date_dimension_df["Date"].dt.strftime('%b')
date_dimension_df["MonthName_Long"] = date_dimension_df["Date"].dt.strftime('%B')
date_dimension_df["CY_WeekOfYear"] = date_dimension_df["Date"].dt.isocalendar().week

# Calendar period boundaries (dates only)
date_dimension_df["StartOfMonth"] = date_dimension_df["Date"].values.astype('datetime64[M]').astype('datetime64[D]')
date_dimension_df["EndOfMonth"] = (date_dimension_df["StartOfMonth"] + pd.offsets.MonthEnd(1)).dt.date
date_dimension_df["StartOfWeek"] = (date_dimension_df["Date"] - pd.to_timedelta(date_dimension_df["DayOfWeek"], unit='d')).dt.date
date_dimension_df["EndOfWeek"] = (pd.to_datetime(date_dimension_df["StartOfWeek"]) + pd.Timedelta(days=6)).dt.date
date_dimension_df["CY_StartOfQuarter"] = date_dimension_df["Date"].dt.to_period("Q").apply(lambda p: p.start_time.date())
date_dimension_df["CY_EndOfQuarter"] = date_dimension_df["Date"].dt.to_period("Q").apply(lambda p: p.end_time.date())

# Quarter start/end day of week (CY)
date_dimension_df["CY_StartOfQuarter_DayOfWeek"] = pd.to_datetime(date_dimension_df["CY_StartOfQuarter"]).dt.dayofweek
date_dimension_df["CY_EndOfQuarter_DayOfWeek"] = pd.to_datetime(date_dimension_df["CY_EndOfQuarter"]).dt.dayofweek

# Composite calendar keys
date_dimension_df["CY_YearMonth"] = date_dimension_df["Date"].dt.strftime("%Y/%m")
date_dimension_df["CY_YearQuarter"] = date_dimension_df["CY_Year"].astype(str) + "/" + date_dimension_df["CY_Quarter"]

# Fiscal (FS) info
date_dimension_df["FS_Year"] = date_dimension_df["Date"].apply(lambda d: d.year if d.month >= 4 else d.year - 1)
date_dimension_df["FS_Quarter"] = date_dimension_df["Date"].apply(
    lambda d: "Q" + str((((((d.month - 4) % 12) + 1) - 1) // 3) + 1)
)
date_dimension_df["FS_Month"] = date_dimension_df["Date"].apply(lambda d: ((d.month - 4) % 12) + 1)

# Fiscal Month Name Short
fiscal_month_names = ["Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec", "Jan", "Feb", "Mar"]
date_dimension_df["FS_MonthName_Short"] = date_dimension_df["FS_Month"].apply(lambda x: fiscal_month_names[x-1])

# FS start of quarter helper function
def start_of_fs_quarter(date):
    fiscal_year = date.year if date.month >= 4 else date.year - 1
    fiscal_month = ((date.month - 4) % 12) + 1
    quarter_start_month = 4 + ((fiscal_month - 1) // 3) * 3
    if quarter_start_month > 12:
        fiscal_year += 1
        quarter_start_month -= 12
    return pd.Timestamp(fiscal_year, quarter_start_month, 1).date()

date_dimension_df["FS_StartOfQuarter"] = date_dimension_df["Date"].apply(start_of_fs_quarter)
date_dimension_df["FS_EndOfQuarter"] = (pd.to_datetime(date_dimension_df["FS_StartOfQuarter"]) + pd.offsets.QuarterEnd(startingMonth=3)).dt.date

# Quarter start/end day of week (FS)
date_dimension_df["FS_StartOfQuarter_DayOfWeek"] = pd.to_datetime(date_dimension_df["FS_StartOfQuarter"]).dt.dayofweek
date_dimension_df["FS_EndOfQuarter_DayOfWeek"] = pd.to_datetime(date_dimension_df["FS_EndOfQuarter"]).dt.dayofweek

# FS Week of Year (week 1 starts April 1)
def fiscal_week_of_year(date):
    fiscal_year_start = pd.Timestamp(date.year if date.month >= 4 else date.year - 1, 4, 1)
    return ((date - fiscal_year_start).days // 7) + 1

date_dimension_df["FS_WeekOfYear"] = date_dimension_df["Date"].apply(fiscal_week_of_year)

# Composite FS keys
date_dimension_df["FS_YearMonth"] = date_dimension_df["FS_Year"].astype(str) + "/" + date_dimension_df["FS_Month"].apply(lambda m: f"{m:02}")
date_dimension_df["FS_YearQuarter"] = date_dimension_df["FS_Year"].astype(str) + "/" + date_dimension_df["FS_Quarter"].astype(str)
date_dimension_df["FS_YearSpan"] = date_dimension_df["FS_Year"].astype(str) + "/" + (date_dimension_df["FS_Year"] + 1).astype(str).str[-2:]

# Days in month
date_dimension_df["DaysInMonth"] = date_dimension_df["Date"].dt.days_in_month

# Last Working Day of Month (Mon-Fri)
def last_working_day(date):
    last_day = date + pd.offsets.MonthEnd(0)
    if last_day.weekday() == 5:  # Saturday
        return last_day - pd.Timedelta(days=1)
    elif last_day.weekday() == 6:  # Sunday
        return last_day - pd.Timedelta(days=2)
    else:
        return last_day

date_dimension_df["LastWorkingDayOfMonth"] = date_dimension_df["Date"].apply(last_working_day)

# ISO calendar fields
iso_calendar = date_dimension_df["Date"].dt.isocalendar()
date_dimension_df["ISO_Year"] = iso_calendar.year
date_dimension_df["ISO_WeekOfYear"] = iso_calendar.week
date_dimension_df["ISO_Weekday"] = iso_calendar.day

# ------------------------
# Convert all date columns to date-only (no time)
date_cols = [
    "Date", "StartOfMonth", "EndOfMonth", "StartOfWeek", "EndOfWeek",
    "CY_StartOfQuarter", "CY_EndOfQuarter", "FS_StartOfQuarter", "FS_EndOfQuarter",
    "LastWorkingDayOfMonth"
]
for col in date_cols:
    date_dimension_df[col] = pd.to_datetime(date_dimension_df[col]).dt.date

# ------------ BANK HOLIDAYS JOIN ---------------
# Assume bank_holidays_df is your DataFrame with columns 'date' and 'title'
bank_holidays_df['date'] = pd.to_datetime(bank_holidays_df['date']).dt.date

# Merge bank holidays onto date_dimension_df
date_dimension_df = date_dimension_df.merge(
    bank_holidays_df.rename(columns={'date': 'Date', 'title': 'BankHolidayName'}),
    on='Date',
    how='left'
)
date_dimension_df['IsBankHoliday'] = date_dimension_df['BankHolidayName'].notna()

# ------------ SCHOOL HOLIDAYS JOIN ---------------
# Assume school_holidays_df has columns 'closing_date', 'opening_date', and 'Holiday'
school_holidays_df['closing_date'] = pd.to_datetime(school_holidays_df['closing_date']).dt.date
school_holidays_df['opening_date'] = pd.to_datetime(school_holidays_df['opening_date']).dt.date

# Function to check if a date is within any school holiday range
def is_school_holiday(date, holidays_df):
    return ((holidays_df['closing_date'] <= date) & (holidays_df['opening_date'] >= date)).any()

# Function to get concatenated school holiday names for a date
def get_school_holiday_names(date, holidays_df):
    names = holidays_df.loc[
        (holidays_df['closing_date'] <= date) & (holidays_df['opening_date'] >= date), 'Holiday'
    ].tolist()
    return ', '.join(names) if names else None

# Apply these functions to the date dimension
date_dimension_df['SchoolHolidayName'] = date_dimension_df['Date'].apply(lambda d: get_school_holiday_names(d, school_holidays_df))
date_dimension_df['IsSchoolHoliday'] = date_dimension_df['Date'].apply(lambda d: is_school_holiday(d, school_holidays_df))

# Final sort and reset index
date_dimension_df = date_dimension_df.sort_values("Date").reset_index(drop=True)

# Display or return your final date dimension DataFrame
display(date_dimension_df)


In [ ]:
# Step 1: Ensure shift pattern is ordered by Rota (1–8)
shift_pattern_df = shift_pattern_df.sort_values('Rota').reset_index(drop=True)

# Step 2: Calculate the Rota_Day (1–8) based on 8-day cycle from 2000-01-01
rota_start_date = pd.to_datetime("2000-01-01").date()
date_dimension_df["Rota_Day"] = ((date_dimension_df["Date"] - rota_start_date).apply(lambda x: x.days) % 8) + 1


# Step 3: Merge the shift pattern data into the date dimension
date_dimension_df = date_dimension_df.merge(
    shift_pattern_df,
    left_on="Rota_Day",
    right_on="Rota",
    how="left"
)

# Optional: drop duplicate 'Rota' column if needed
date_dimension_df = date_dimension_df.drop(columns=["Rota"])

# Reorder for clarity if desired
# columns = ["Date", "Rota_Day", "Pattern_BlueShift", "Pattern_GreenShift", ...] + other date columns
# date_dimension_df = date_dimension_df[columns]

display(date_dimension_df)

In [ ]:
import pandas as pd

def map_shift_to_phase(shift_str):
    shift_str = str(shift_str).lower()
    if "day" in shift_str:
        return 0
    elif "night" in shift_str:
        return 1
    else:
        # Rota or other = 2
        return 2

def compute_tour_counts_with_reset(df, pattern_cols):
    df = df.copy()
    df['Date'] = pd.to_datetime(df['Date'])
    
    # Fiscal year starting April 1
    df['Fiscal_Year'] = df['Date'].apply(lambda d: d.year if d.month >= 4 else d.year - 1)

    df = df.sort_values("Date").reset_index(drop=True)

    for col in pattern_cols:
        fiscal_col = "FS_" + col.split("_")[1].replace("Shift", "") + "_Tour_Count"
        global_col = fiscal_col.replace("FS_", "AL_Global_")

        df[fiscal_col] = 0
        df[global_col] = 0

        current_fiscal_year = None
        fiscal_tour_index = 0
        global_tour_index = 3
        day_counter = 0
        tour_active = False

        fiscal_counts = []
        global_counts = []

        for idx, row in df.iterrows():
            date = row["Date"]
            fy = row["Fiscal_Year"]
            phase = map_shift_to_phase(row[col])

            if fy != current_fiscal_year:
                # Reset fiscal counter
                current_fiscal_year = fy
                fiscal_tour_index = 0
                day_counter = 0
                tour_active = False

            if not tour_active:
                if phase == 0:
                    tour_active = True
                    fiscal_tour_index += 1
                    global_tour_index += 1
                    day_counter = 1
            else:
                day_counter += 1
                if day_counter == 8:
                    fiscal_tour_index += 1
                    global_tour_index += 1
                    day_counter = 0

            fiscal_counts.append(fiscal_tour_index)
            global_counts.append(global_tour_index)

        df[fiscal_col] = fiscal_counts
        df[global_col] = global_counts

    df.drop("Fiscal_Year", axis=1, inplace=True)
    return df



# Usage
pattern_columns = ["Pattern_BlueShift", "Pattern_GreenShift", "Pattern_RedShift", "Pattern_WhiteShift"]

date_dimension_df = compute_tour_counts_with_reset(date_dimension_df, pattern_columns)


In [ ]:
date_dimension_df.query("Date == '2026-04-01'")[["Date", "AL_Global_Red_Tour_Count"]]


In [ ]:
# Get count of days in each Red tour
red_tour_sizes = (
    date_dimension_df
    .groupby(["FS_Year", "FS_Red_Tour_Count"])
    .size()
    .reset_index(name="DayCount")
)

# Filter only problematic ones (not equal to 8)
problem_red_tours = red_tour_sizes[red_tour_sizes["DayCount"] != 8]

display(problem_red_tours)


In [ ]:
display(date_dimension_df)

In [ ]:
for col in ['Date', 'StartOfMonth', 'LastWorkingDayOfMonth']:
    date_dimension_df[col] = pd.to_datetime(date_dimension_df[col]).dt.strftime('%Y-%m-%d')

In [ ]:
import pandas as pd
from datetime import datetime

def generate_first_group_mapping_df(start_year=2010, years_ahead=45, years_back=11):
    pattern = ['A', 'J', 'B', 'K', 'C', 'L', 'D', 'G', 'E', 'H', 'F', 'I']
    current_year = datetime.now().year
    end_year = current_year + years_ahead
    start_year_back = start_year - years_back  # e.g. 2010 - 11 = 1999

    # Combine years backward and forward into one list
    years = list(range(start_year_back, start_year)) + list(range(start_year, end_year + 1))

    data = [
        {"FS_Year": year, "First_Group": pattern[(year - start_year) % len(pattern)]}
        for year in years
    ]

    return pd.DataFrame(data)

# Create the DataFrame with backwards extension to 1999
first_group_df = generate_first_group_mapping_df()

display(first_group_df)


In [ ]:
# Define base and extras
base_pattern = ['A', 'J', 'B', 'K', 'C', 'L', 'D', 'G', 'E', 'H', 'F', 'I']
full_base = base_pattern * 3  # 36 total
extras = ['Ex1', 'Ex2', 'Ex3', 'Ex4']

# Map FS_Year -> pattern start index
start_index_map = {
    row['FS_Year']: base_pattern.index(row['First_Group'])
    for _, row in first_group_df.iterrows()
}

# Function to return group for a tour count
def get_group_letter_with_year(fs_year, tour_count):
    if pd.isna(tour_count) or tour_count == 0:
        return None

    start_idx = start_index_map.get(fs_year, 0)
    # Rotate the full 36-tour pattern
    rotated_36 = full_base[start_idx:] + full_base[:start_idx]
    full_shifted = rotated_36 + extras  # 40 total

    tour_index = int(tour_count) - 1
    if 0 <= tour_index < len(full_shifted):
        return full_shifted[tour_index]
    else:
        return None  # Beyond 40 tours


In [ ]:
import pandas as pd
from datetime import datetime

# Year-start pattern (defines who goes first each year)
year_start_pattern = ['A', 'J', 'B', 'K', 'C', 'L', 'D', 'G', 'E', 'H', 'F', 'I']

# Alphabetical order used inside year
alphabetical_pattern = ['A','B','C','D','E','F','G','H','I','J','K','L']

total_blocks = 46
ex_insert_map = {
    33: 'Ex1',
    34: 'Ex2',
    43: 'Ex3',
    44: 'Ex4'
}

# Offsets relative to first index for the in-year pattern
inside_year_offsets = [
    0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 9, 10, 10, 11, 11, 0, 0, 1, 1, 2, 2,
    3, 4, 5, 6, 7, 8, 9, 10, 11, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 0
]

def get_first_group(fs_year, base_year=2010):
    return year_start_pattern[(fs_year - base_year) % len(year_start_pattern)]

def generate_leave_pattern(fs_year, base_year=2010):
    first_group = get_first_group(fs_year, base_year)
    first_index = alphabetical_pattern.index(first_group)

    normal_blocks = [
        alphabetical_pattern[(first_index + offset) % len(alphabetical_pattern)]
        for offset in inside_year_offsets
    ]

    full_pattern = []
    i_normal = 0
    for i in range(total_blocks):
        if i in ex_insert_map:
            full_pattern.append(ex_insert_map[i])
        else:
            full_pattern.append(normal_blocks[i_normal])
            i_normal += 1
    return full_pattern

def generate_annual_leave_df(start_year=2000, years_ahead=None, base_year=2010):
    if years_ahead is None:
        current_year = datetime.now().year
        years_ahead = current_year + 45 - start_year

    years = range(start_year, start_year + years_ahead + 1)
    rows = []
    global_block_index = 1  # ← Global continuous counter

    for year in years:
        pattern_for_year = generate_leave_pattern(year, base_year)
        for idx, group in enumerate(pattern_for_year, start=1):
            rows.append({
                'FS_Year': year,
                'Block_Index': idx,               # Resets yearly
                'Global_Block_Index': global_block_index,  # Continuous
                'Group': group
            })
            global_block_index += 1

    return pd.DataFrame(rows)

# Run it
annual_leave_schedule_df = generate_annual_leave_df(start_year=2000, base_year=2010)
display(annual_leave_schedule_df)


In [ ]:
testing_df = annual_leave_schedule_df

display(testing_df[testing_df["FS_Year"].isin([2024, 2025, 2026, 2027])])


In [ ]:
# Color mappings
watch_colors = ["White", "Red", "Green", "Blue"]

# Start with the base date dimension DataFrame
joined_df = date_dimension_df.copy()

for color in watch_colors:
    # Temporary rename: Group → AL_<Color>_Block
    tmp = (
        annual_leave_schedule_df
        .rename(columns={"Group": f"AL_Block_{color}"})
        [["FS_Year", "Block_Index", f"AL_Block_{color}"]]
    )

    # Join using FS_Year and the appropriate FS_<Color>_Tour_Count
    joined_df = joined_df.merge(
        tmp,
        how="left",
        left_on=["FS_Year", f"FS_{color}_Tour_Count"],
        right_on=["FS_Year", "Block_Index"]
    ).drop(columns=["Block_Index"])

# Final result
display(joined_df)


In [ ]:
use_global = True  # or False to use FS counts

In [ ]:
# Color mappings
watch_colors = ["White", "Red", "Green", "Blue"]

# Start with the base date dimension DataFrame
joined_df = date_dimension_df.copy()

for color in watch_colors:
    # Determine tour count column name
    join_col = f"AL_{'Global' if use_global else 'FS'}_{color}_Tour_Count"
    
    # Determine right-side (schedule) join key
    block_index_col = "Global_Block_Index" if use_global else "Block_Index"

    # Prepare the AL schedule table
    tmp = (
        annual_leave_schedule_df
        .rename(columns={"Group": f"AL_Block_{color}"})
        [[block_index_col] + ([] if use_global else ["FS_Year"]) + [f"AL_Block_{color}"]]
    )

    # Determine join keys
    left_keys = [join_col] if use_global else ["FS_Year", join_col]
    right_keys = [block_index_col] if use_global else ["FS_Year", block_index_col]

    # Merge
    joined_df = joined_df.merge(
        tmp,
        how="left",
        left_on=left_keys,
        right_on=right_keys
    ).drop(columns=[block_index_col])

# Final result
display(joined_df)

In [ ]:
for col in ['Date', 'StartOfMonth', 'LastWorkingDayOfMonth']:
    joined_df[col] = pd.to_datetime(joined_df[col]).dt.strftime('%Y-%m-%d')

import shutil
import os

table_path = '/lakehouse/default/Tables/Date_Table'

if os.path.exists(table_path):
    shutil.rmtree(table_path)  # WARNING: Deletes all files in the folder

write_deltalake(
    table_path,
    joined_df,
    mode='overwrite',
    schema_mode='merge',
    engine='rust',
    storage_options=storage_options
)
